In [7]:
import spacy
from spacy import displacy
import random
from spacy.util import minibatch, compounding

nlp = spacy.load("C:/Users/LABS/Documents/tensor/new_model")

if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe("ner")
    
TRAIN_DATA = [
    ( "O meu nome é José Carlos e tenho 32 anos de idade", { "entities": [ (13,24, "NOME"), (33,35, "IDADE")] } )
]
    
# add labels
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# get names of other pipes to disable them during training
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
with nlp.disable_pipes(*other_pipes):  # only train NER
    # reset and initialize the weights randomly – but only if we're
    # training a new model
    for itn in range(1,5):
        random.shuffle(TRAIN_DATA)
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(
                texts,  # batch of texts
                annotations,  # batch of annotations
                drop=0.2,  # dropout - make it harder to memorise data
                losses=losses,
            )
        print("Losses", losses)
        
nlp.to_disk("C:/Users/LABS/Documents/tensor/new_model")

Losses {'ner': 1.1596684313822105}
Losses {'ner': 0.9107353633343372}
Losses {'ner': 11.957296199477698}
Losses {'ner': 12.12710815884567}
